# Traveling Salesman Problem (TSP)

**Objectives**

- Introduce students to a real world problem solved by OR practitioners
- Demonstrate the use of heuristics to obtain good solutions to optimization problems
- Give students an appreciation of the difficulty of solving optimization problems exactly

**Reading:** Read Handout 2 on the traveling salesman problem.

**Brief description:** Finding an optimal solution to a Traveling Salesman Problem, and proving that it is, in fact, an optimal solution, is a difficult task. In practice, when a feasible solution to a difficult problem needs to be provided quickly, one often resorts to using heuristics, i.e., procedures for generating feasible solutions, or improving existing ones, that can be executed quickly and, hopefully, produce a pretty good result. In this lab we will consider several such heuristic procedures for the TSP.

<font color='blue'> <b>Solutions are shown blue.</b> </font> <br>
<font color='red'> <b>Instuctor comments are shown in red.</b> </font>

<font color='red'>A tool we might want to use: [TSP DIY](https://www.math.uwaterloo.ca/tsp/app/diy.html)  </font>

<font color='red'> Some explanation of what imports are here.</font>

In [1]:
# imports -- don't forget to run this cell
from tsp import *
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
nodes, G = tsp_grid_instance(20,20,manhattan=False)
tour = nearest_neighbor(G)
plot_tour(nodes, G, tour)

In [3]:
nodes, G = tsp_grid_instance(6,8,manhattan=False)
plot_tsp_heuristic(nodes, G, heuristic='furthest_insertion', initial=[0,len(G)-1,0])

In [4]:
n = 50
random_neighbor_total = 0
nearest_neighbor_total = 0
nearest_insertion_total = 0
furthest_insertion_total = 0
for i in range(n):
    nodes, G = tsp_grid_instance(6,8,True)
    random_neighbor_total += tour_cost(G, random_neighbor(G))
    nearest_neighbor_total += tour_cost(G, nearest_neighbor(G))
    nearest_insertion_total += tour_cost(G, nearest_insertion(G))
    furthest_insertion_total += tour_cost(G, furthest_insertion(G))
print("Heuristic Averages:")
print("Random Neighbor: %s" % (random_neighbor_total / n))
print("Nearest Neighbor: %s" % (nearest_neighbor_total / n))
print("Nearest Insertion: %s" % (nearest_insertion_total / n))
print("Furthest Insertion: %s" % (furthest_insertion_total / n))

Heuristic Averages:
Random Neighbor: 223.08
Nearest Neighbor: 63.6
Nearest Insertion: 55.68
Furthest Insertion: 56.12
